In [1]:
# Import necessary libraries
import sys
import cv2
import tensorflow as tf
from PyQt5 import QtCore, QtGui, QtWidgets
from PyQt5.QtGui import QImage, QPixmap
from PyQt5.QtWidgets import QApplication, QFileDialog
import numpy as np
from functions import get_image_embeddings, preprocess_to_lfw  # Assuming you have these helper functions

from PyQt5.QtCore import QTimer


class FaceRecognitionGUI(QtWidgets.QMainWindow):
    def __init__(self):
        super().__init__()
        # other initializations...
        self.webcam_timer = QTimer(self)
        self.webcam_timer.timeout.connect(self.update_webcam)  # Connect the timer to the update function

        self.setWindowTitle("Face Recognition")
        self.setGeometry(100, 100, 900, 666)

        # Create central widget and set layout
        self.centralwidget = QtWidgets.QWidget(self)
        self.setCentralWidget(self.centralwidget)

        self.image_verification = QtWidgets.QTabWidget(self.centralwidget)
        self.image_verification.setGeometry(QtCore.QRect(10, 10, 881, 611))

        # Image Verification Tab (First Tab)
        self.widget = QtWidgets.QWidget(self)
        self.image_verification.addTab(self.widget, "Image Verification")

        # Initialize Models combo box for the first tab
        self.Models = QtWidgets.QComboBox(self.widget)
        self.Models.setGeometry(QtCore.QRect(310, 30, 231, 22))
        self.Models.setFont(QtGui.QFont('Arial', 10, QtGui.QFont.Bold))
        self.Models.addItem("ResNet50")
        self.Models.addItem("mobilenetv2")

        self.label = QtWidgets.QLabel("Model Selection", self.widget)
        self.label.setGeometry(QtCore.QRect(310, 10, 221, 20))
        self.label.setAlignment(QtCore.Qt.AlignCenter)

        # Now connect the signal to the slot after initialization
        self.Models.currentIndexChanged.connect(self.on_model_change)

        # Other widgets for Image Verification tab
        self.first_image = QtWidgets.QLabel(self.widget)
        self.first_image.setGeometry(QtCore.QRect(30, 170, 361, 351))

        self.second_image_label = QtWidgets.QLabel(self.widget)
        self.second_image_label.setGeometry(QtCore.QRect(470, 170, 361, 351))

        self.upload_first_image = QtWidgets.QPushButton("Upload Reference Image", self.widget)
        self.upload_first_image.setGeometry(QtCore.QRect(30, 540, 361, 28))
        self.upload_first_image.clicked.connect(self.upload_reference_image)

        self.upload_second_image_button = QtWidgets.QPushButton("Upload Second Image", self.widget)
        self.upload_second_image_button.setGeometry(QtCore.QRect(470, 540, 361, 28))
        self.upload_second_image_button.clicked.connect(self.upload_second_image)

        self.verify = QtWidgets.QPushButton("Verify", self.widget)
        self.verify.setGeometry(QtCore.QRect(370, 60, 93, 28))
        self.verify.clicked.connect(self.verify_images)

        self.results_text = QtWidgets.QTextEdit(self.widget)
        self.results_text.setGeometry(QtCore.QRect(193, 90, 441, 41))

        # Webcam Verification Tab (Second Tab)
        self.tab_2 = QtWidgets.QWidget(self)
        self.image_verification.addTab(self.tab_2, "Webcam Verification")

        self.ref_image_label_2 = QtWidgets.QLabel(self.tab_2)
        self.ref_image_label_2.setGeometry(QtCore.QRect(30, 170, 361, 351))

        self.webcam_label = QtWidgets.QLabel(self.tab_2)
        self.webcam_label.setGeometry(QtCore.QRect(470, 170, 361, 351))

        self.upload_ref_image_btn_2 = QtWidgets.QPushButton("Upload Reference Image", self.tab_2)
        self.upload_ref_image_btn_2.setGeometry(QtCore.QRect(30, 540, 361, 28))
        self.upload_ref_image_btn_2.clicked.connect(self.upload_reference_image_2)

        # Initialize Models combo box for the second tab
        self.Models_2 = QtWidgets.QComboBox(self.tab_2)
        self.Models_2.setGeometry(QtCore.QRect(310, 30, 231, 22))
        self.Models_2.setFont(QtGui.QFont('Arial', 10, QtGui.QFont.Bold))
        self.Models_2.addItem("ResNet50")
        self.Models_2.addItem("mobilenetv2")

        self.webcam_btn = QtWidgets.QPushButton("Start Webcam", self.tab_2)
        self.webcam_btn.setGeometry(QtCore.QRect(470, 540, 180, 28))
        self.webcam_btn.clicked.connect(self.toggle_webcam)

        self.verify_webcam_btn = QtWidgets.QPushButton("Verify Webcam", self.tab_2)
        self.verify_webcam_btn.setGeometry(QtCore.QRect(651, 540, 180, 28))
        self.verify_webcam_btn.clicked.connect(self.verify_webcam)

        self.webcam_results_text = QtWidgets.QTextEdit(self.tab_2)
        self.webcam_results_text.setGeometry(QtCore.QRect(193, 90, 441, 41))

        # Now connect the signal to the slot for the second combo box
        self.Models_2.currentIndexChanged.connect(self.on_model_change)

        # Initialize webcam capture
        self.webcam = None
        self.current_model = None
        self.threshold = 0.3
        self.model_paths = {
            "ResNet50": "resnet50.keras",
            "mobilenetv2": "mobilenetv2.keras",
        }
        self.cached_image1_path = None
        self.cached_image2_path = None
        self.cached_encoding1 = None
        self.cached_encoding2 = None
        self.ref_image_path = None

    # Remaining methods (load_model, upload_reference_image, etc.) should remain the same

    def load_model(self, model_name):
        if model_name in self.model_paths:
            self.current_model = tf.keras.models.load_model(self.model_paths[model_name])
            print(f"Model {model_name} loaded successfully.")
    
            # Clear cached data to avoid conflicts when switching models
            self.cached_image1_path = None
            self.cached_image2_path = None
            self.cached_encoding1 = None
            self.cached_encoding2 = None
            self.ref_image_path = None
    def upload_reference_image(self):
        file_name, _ = QFileDialog.getOpenFileName(
            self, "Open Reference Image", "", "Image Files (*.png *.jpg *.jpeg)"
        )
        if file_name:
            image = cv2.imread(file_name)
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
            faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

            if len(faces) > 0:
                x, y, w, h = sorted(faces, key=lambda face: face[2] * face[3], reverse=True)[0]
                cv2.rectangle(image, (x, y), (x+w, y+h), (255, 0, 0), 2)

            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            height, width, channel = image.shape
            bytes_per_line = 3 * width
            q_image = QImage(image.data, width, height, bytes_per_line, QImage.Format_RGB888)
            pixmap = QPixmap.fromImage(q_image)
            
            self.ref_image_path = file_name
            self.first_image.setPixmap(pixmap.scaled(400, 400, QtCore.Qt.KeepAspectRatio))
    def on_model_change(self):
        # Check which combo box is calling this function
        if self.sender() == self.Models:  # For Image Verification tab
            selected_model = self.Models.currentText()
            print(f"Selected Model from Image Verification: {selected_model}")
            self.load_model(selected_model)
        elif self.sender() == self.Models_2:  # For Webcam Verification tab
            selected_model = self.Models_2.currentText()
            print(f"Selected Model from Webcam Verification: {selected_model}")
            self.load_model(selected_model)

    def upload_reference_image_2(self):
        file_name, _ = QFileDialog.getOpenFileName(
            self, "Open Reference Image", "", "Image Files (*.png *.jpg *.jpeg)"
        )
        if file_name:
            image = cv2.imread(file_name)
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
            faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

            if len(faces) > 0:
                x, y, w, h = sorted(faces, key=lambda face: face[2] * face[3], reverse=True)[0]
                cv2.rectangle(image, (x, y), (x+w, y+h), (255, 0, 0), 2)

            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            height, width, channel = image.shape
            bytes_per_line = 3 * width
            q_image = QImage(image.data, width, height, bytes_per_line, QImage.Format_RGB888)
            pixmap = QPixmap.fromImage(q_image)
            
            self.ref_image_path = file_name
            self.ref_image_label_2.setPixmap(pixmap.scaled(400, 400, QtCore.Qt.KeepAspectRatio))
    def upload_second_image(self):
        options = QFileDialog.Options()
        file, _ = QFileDialog.getOpenFileName(self, "Upload Second Image", "", "Images (*.png *.xpm *.jpg *.jpeg);;All Files (*)", options=options)
        
        if file:
            # Store the second image path
            self.second_image_path = file
            self.second_image_label.setPixmap(QtGui.QPixmap(file).scaled(self.second_image_label.size(), QtCore.Qt.KeepAspectRatio))
    
            # Use the correct 'file' variable, not 'file_name'
            image = cv2.imread(file)  # Corrected variable name
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
            faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
            if len(faces) > 0:
                # Sort faces by the largest area and draw a rectangle around the largest one
                x, y, w, h = sorted(faces, key=lambda face: face[2] * face[3], reverse=True)[0]
                cv2.rectangle(image, (x, y), (x+w, y+h), (255, 0, 0), 2)
    
            # Convert the image to RGB format for display
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            height, width, channel = image.shape
            bytes_per_line = 3 * width
            q_image = QImage(image.data, width, height, bytes_per_line, QImage.Format_RGB888)
            pixmap = QPixmap.fromImage(q_image)
    
            # Update the label with the processed image
            self.second_image_label.setPixmap(pixmap.scaled(400, 400, QtCore.Qt.KeepAspectRatio))
    

    def verify_images(self):
        # Check if both images are uploaded
        if not hasattr(self, 'ref_image_path') or self.ref_image_path is None:
            self.results_text.setText("Please upload a reference image before verifying.")
            return
    
        if not hasattr(self, 'second_image_path') or self.second_image_path is None:
            self.results_text.setText("Please upload a second image before verifying.")
            return
    
        # Ensure the model is loaded
        if self.current_model is None:
            self.load_model(self.Models.currentText())
    
        # Load and process images
        image1 = cv2.imread(self.ref_image_path)
        image2 = cv2.imread(self.second_image_path)
    
        # Check if embeddings are already cached for the first image
        if not hasattr(self, 'cached_image1_path') or self.cached_image1_path != self.ref_image_path:
            try:
                self.cached_encoding1 = get_image_embeddings(self.current_model, image1, False)
                self.cached_image1_path = self.ref_image_path
            except ValueError as e:
                if str(e) == "No faces detected in the image.":
                    self.results_text.setText("No face detected in the first image.")
                    return
                else:
                    self.results_text.setText("An error occurred during image verification.")
                    return
    
        # Check if embeddings are already cached for the second image
        if not hasattr(self, 'cached_image2_path') or self.cached_image2_path != self.second_image_path:
            try:
                self.cached_encoding2 = get_image_embeddings(self.current_model, image2, False)
                self.cached_image2_path = self.second_image_path
            except ValueError as e:
                if str(e) == "No faces detected in the image.":
                    self.results_text.setText("No face detected in the second image.")
                    return
                else:
                    self.results_text.setText("An error occurred during image verification.")
                    return
    
        # Calculate similarity metrics
        distance = tf.norm(self.cached_encoding1 - self.cached_encoding2).numpy()
        cosine_similarity = tf.tensordot(self.cached_encoding1, self.cached_encoding2, axes=1).numpy() / (
            tf.norm(self.cached_encoding1).numpy() * tf.norm(self.cached_encoding2).numpy()
        )
    
        # Display the results

        results = f"{'Match:':<20}{'Yes' if cosine_similarity > self.threshold else 'No'}"
        self.results_text.setText(results)

    def toggle_webcam(self):
        if self.webcam is None:
            self.webcam = cv2.VideoCapture(0)
            self.webcam_btn.setText("Stop Webcam")
            self.webcam_timer.start(30)
        else:
            self.webcam_timer.stop()
            self.webcam.release()
            self.webcam = None
            self.webcam_btn.setText("Start Webcam")
            self.webcam_label.clear()

    def update_webcam(self):
        ret, frame = self.webcam.read()
        if ret:
            self.current_frame = frame.copy()  # Copy for processing purposes
            display_frame = frame.copy()  # Copy for display purposes
            gray = cv2.cvtColor(display_frame, cv2.COLOR_BGR2GRAY)
            face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
            faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

            if len(faces) > 0:
                x, y, w, h = sorted(faces, key=lambda face: face[2] * face[3], reverse=True)[0]
                cv2.rectangle(display_frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

            display_frame = cv2.cvtColor(display_frame, cv2.COLOR_BGR2RGB)
            height, width, channel = display_frame.shape
            bytes_per_line = 3 * width
            q_image = QImage(display_frame.data, width, height, bytes_per_line, QImage.Format_RGB888)
            pixmap = QPixmap.fromImage(q_image)
            self.webcam_label.setPixmap(pixmap.scaled(400, 400, QtCore.Qt.KeepAspectRatio))
    def upload_reference_image_2_function(self):
        file_name, _ = QFileDialog.getOpenFileName(
            self, "Open Reference Image", "", "Image Files (*.png *.jpg *.jpeg)"
        )
        if file_name:
            # Load the image
            image = cv2.imread(file_name)
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
            # Load OpenCV's pre-trained face detector
            face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    
            # Detect faces
            faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
            if len(faces) > 0:
                # Sort faces by size (area) and get the largest one
                x, y, w, h = sorted(faces, key=lambda face: face[2] * face[3], reverse=True)[0]
                cv2.rectangle(image, (x, y), (x+w, y+h), (255, 0, 0), 2)
    
            # Convert the image to QPixmap for display
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            height, width, channel = image.shape
            bytes_per_line = 3 * width
            q_image = QImage(image.data, width, height, bytes_per_line, QImage.Format_RGB888)
            pixmap = QPixmap.fromImage(q_image)
    
            self.first_image_2.setPixmap(pixmap.scaled(400, 400, QtCore.Qt.KeepAspectRatio))
            self.ref_image_path = file_name  # Store the path of the reference image

    def verify_webcam(self):
        # Ensure that the required attributes are present
        if not hasattr(self, 'ref_image_path') or not hasattr(self, 'current_frame'):
            self.webcam_results_text.setText("Reference image or webcam frame not available!")
            return
    
        # Load the model if it hasn't been loaded yet
        if self.current_model is None:
            self.load_model(self.Models.currentText())
    
        # Check if reference image is valid
        ref_image = cv2.imread(self.ref_image_path)
        if ref_image is None:
            self.webcam_results_text.setText("Failed to load reference image!")
            return
    
        # Check if current frame from webcam is valid
        if self.current_frame is None:
            self.webcam_results_text.setText("No frame captured from webcam!")
            return
    
        try:
            # Get embeddings for the reference image and webcam frame
            ref_encoding = get_image_embeddings(self.current_model, ref_image, False)
            webcam_encoding = get_image_embeddings(self.current_model, self.current_frame, False)
    
            # Compute distance (Euclidean distance)
            distance = tf.norm(ref_encoding - webcam_encoding).numpy()
    
            # Compute cosine similarity
            cosine_similarity = tf.tensordot(ref_encoding, webcam_encoding, axes=1).numpy() / (
                tf.norm(ref_encoding).numpy() * tf.norm(webcam_encoding).numpy()
            )
    
            # Format the results
            results = f"{'Match:':<20}{'Yes' if cosine_similarity > self.threshold else 'No'}"
    
            # Update the text box with the results
            self.webcam_results_text.setText(results)
    
        except ValueError as e:
            # Handle specific error when no face is detected
            if str(e) == "No faces detected in the image.":
                self.webcam_results_text.setText("No face detected in the reference or webcam image.")
            else:
                self.webcam_results_text.setText(f"An error occurred during image verification: {str(e)}")
    
        except Exception as e:
            # General exception handling
            self.webcam_results_text.setText(f"Unexpected error: {str(e)}")

    def closeEvent(self, event):
        if self.webcam is not None:
            self.webcam.release()
        event.accept()

if __name__ == '__main__':
    app = QApplication(sys.argv)
    window = FaceRecognitionGUI()
    window.show()
    sys.exit(app.exec_())


Selected Model from Webcam Verification: mobilenetv2
Model mobilenetv2 loaded successfully.
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 967ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 826ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 780ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 847ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 799ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 809ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 986ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 806ms/step
Selected Model from Webcam Verification: ResNet50
Model ResNet50 loaded successfully.
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Selected Model from Webcam Verification: mobilenetv2
Model mobilenetv2 loaded successfully.
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 792ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 811ms/step
Selected Model from Webcam Verification: ResNet50
Model ResNet50 loaded successfully.
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Selected Model from Webcam Verificatio

SystemExit: 0

C:\Users\nadee\anaconda3\envs\session2\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
